In [2]:

# import required classes
import numpy as np
import os
from skimage.io import imread, imsave
from PIL import Image, ImageDraw, ImageFont
from keras.models import load_model
#import imageio.imwrite

#import resnet
import sys
if "E:\\git\\keras-resnet" not in sys.path:
    sys.path.append("E:\\git\\keras-resnet")
    print(sys.path)

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# configure memory allocate for tensorflow backend
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# vertical_model = load_model('E:\\UAV_drone\\training_results\\keras\\resnet18\\model-uav-vertical-50.hdf5')
# horizontal_model = load_model('E:\\UAV_drone\\training_results\\keras\\resnet18\\model-uav-view-28.hdf5')
# latoff_model = load_model('E:\\UAV_drone\\training_results\\keras\\resnet18\\model-uav-offset-27.hdf5')

vertical_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TraningModels\\keras-resnet\\checkpoint\\model-uav-height-44.hdf5')
horizontal_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TraningModels\\keras-resnet\\checkpoint\\model-uav-view_offset-47.hdf5')

Using TensorFlow backend.


['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet']


In [2]:

# f = 'E:\\UAV_drone\\data\\2019-01-04-6groups_120fov_height_normal\\val\\medium\\img_0_0_1546559335497177700.png'
# img = imread(f) 
# img = img[:,:,:3]
# img = img / 255.0
# img = np.reshape(img,[1,512,512,3])
# height_prob = vertical_model.predict(img)[0]
# horizontal_prob = horizontal_model.predict(img)
# offset_prob = horizontal_prob[0][0]
# view_prob = horizontal_prob[1][0]
# print(height_prob)
# print(offset_prob)
# print(view_prob)

In [3]:

FLYING_LOG_DIR = "E:\\UAV_drone\\Logs\\flying_recording_tmp8\\"

if not os.path.exists(FLYING_LOG_DIR):
    os.makedirs(FLYING_LOG_DIR)

In [25]:
def write_probs_on_image(height_prob, offset_prob, view_prob, image_file, output_dir):
    # initialise the drawing context with
    # the image object as background
    img = Image.open(image_file)
    draw = ImageDraw.Draw(img)
    # create font object with the font file and specify
    # desired size
    font = ImageFont.truetype('Roboto-Medium.ttf', size=15)
    
    # OFFSET
    (x, y) = (235, 450)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'Offset: ', fill=color, font=font)
    
    (x, y) = (290, 450)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'C', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(offset_prob[0])), fill=color, font=font)

    (x, y) = (355, 450)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'M', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(offset_prob[1])), fill=color, font=font)

    (x, y) = (420, 450)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'F', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(offset_prob[2])), fill=color, font=font)
    
    # VIEW
    (x, y) = (235, 420)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'View: ', fill=color, font=font)
    
    (x, y) = (290, 420)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'L', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(view_prob[0])), fill=color, font=font)

    (x, y) = (355, 420)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'S', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(view_prob[1])), fill=color, font=font)

    (x, y) = (420, 420)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'R', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(view_prob[2])), fill=color, font=font)
    
    # HEIGHT
    # starting position of the message
    (x, y) = (290, 340)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'Altitude: ', fill=color, font=font)
    
    (x, y) = (355, 320)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'H', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(height_prob[2])), fill=color, font=font)

    (x, y) = (355, 340)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'M', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(height_prob[1])), fill=color, font=font)

    (x, y) = (355, 360)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'L', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(height_prob[0])), fill=color, font=font)

    
    # save the edited image
    prob_file = 'prob_' + os.path.basename(image_file)
    print(prob_file)
    img.save(output_dir + prob_file)

f = 'img_0_0_1546722126943805600.png'
img = imread(f) 
img = img[:,:,:3]
img = img / 255.0
img = np.reshape(img,[1,512,512,3])
height_prob = vertical_model.predict(img)[0]
horizontal_prob = horizontal_model.predict(img)
offset_prob = horizontal_prob[0][0]
view_prob = horizontal_prob[1][0]
print(height_prob)
print(offset_prob)
print(view_prob)
output_dir = ''
write_probs_on_image(height_prob, offset_prob, view_prob, f, output_dir)


[0.01193932 0.9831306  0.00493004]
[6.2860221e-07 6.3337362e-01 3.6662582e-01]
[3.2484628e-02 9.6748406e-01 3.1321571e-05]
prob_img_0_0_1546722126943805600.png


### Sanity test the model

In [4]:
import common_functions as control
from common_types import *
import airsim
from airsim.types import *

import pprint
import time
import math


def transform_input(responses, i, f, recording=False):
   
    response = responses[0]
    img1d_rgba = np.fromstring(response.image_data_uint8, dtype=np.uint8)
    img4d_rgba = img1d_rgba.reshape(512, 512, 4)
    img3d_rbg = img4d_rgba[:, :, :3] #512*512*3
    
    if (recording): 
        img = os.path.join(FLYING_LOG_DIR , 'outfile'+str(i)+'.png')
        imsave(img, img3d_rbg)
    
    img3d_rbg = np.reshape(img3d_rbg,[1,512,512,3])
    img_final = img3d_rbg/255.
    
    height_prob = vertical_model.predict(img_final)[0]
    horizontal_prob = horizontal_model.predict(img_final)
    offset_prob = horizontal_prob[0][0]
    view_prob = horizontal_prob[1][0]
    
    
    if (recording): 
        f.write("{}, VERTICAL: Low {:.3f}, Medium {:.3f}, High {:.3f}\n".format(i, height_prob[0], height_prob[1], height_prob[2]))
        f.write("{}, LATOFF: Left {:.3f},  Center {:.3f}, Right {:.3f}\n".format(i, offset_prob[0], offset_prob[1], offset_prob[2]))
        f.write("{}, VIEW: Left {:.3f},  Straight {:.3f}, Right {:.3f}\n".format(i, view_prob[0], view_prob[1], view_prob[2]))
        
    
    return (height_prob, offset_prob, view_prob)

def getPitchRollYawInRad():
    return airsim.to_eularian_angles(client.simGetGroundTruthKinematics().orientation)

def getZ():
    return client.simGetGroundTruthKinematics().position.z_val

def toDegree(rad):
    return rad*180.0/math.pi

def toRad(angle):
    return angle*math.pi/180.0


def moveByYawPitch(cur_yaw_rad, new_yaw_rad, z, velocity, headless= False, file=None, recording=False): 
    yaw = cur_yaw_rad + new_yaw_rad
    vx = velocity*math.cos(yaw)
    vy = velocity*math.sin(yaw)
    if recording == True:
        file.write('headless {}, vx {:.3f}, vy {:.3f}, z {:.3f}, yaw {}\n'.format(headless, vx, vy, z, -toDegree(new_yaw_rad)))
    
    if headless == True:
        client.moveByVelocityZAsync(vx, vy, z, 0.6, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,-toDegree(new_yaw_rad))).join()
    else:
        client.moveByVelocityZAsync(vx, vy, z, 0.6, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()


['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet', 'E:\\git\\AirSim\\PythonClient']
-- ['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib

In [21]:
mastDir = 'C:\\Users\\hoang\\WorkingSpace\\Unreal Projects\\DroneSimulator_Normal2\\Map_objects_info\\Masts'
playerStartDir = 'C:\\Users\\hoang\\WorkingSpace\\Unreal Projects\\DroneSimulator_Normal2\\Map_objects_info\\PlayerStart'

playerStart = control.readStartLocation(playerStartDir)
print(playerStart)

mastPrefix4 = 'H_mast_ins_brown_4_white_4'
masts = control.readMastLocations(mastDir, playerStart, mastPrefix4)
print(len(masts))

print(masts[0])

# connect to the AirSim simulator
client = airsim.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
# takeoff
client.takeoffAsync().join()
client.moveToZAsync(-1.5, 1).join()
client.simSetCameraOrientation("0", airsim.to_quaternion(-0.7854, 0, 0)) # 45 degree 

XYZ(84859.523 24741.189 1186.079) Yaw(0.0)
H_mast_ins_brown_4_white_4_00
H_mast_ins_brown_4_white_4_01
H_mast_ins_brown_4_white_4_02
H_mast_ins_brown_4_white_4_03
H_mast_ins_brown_4_white_4_04
H_mast_ins_brown_4_white_4_05
H_mast_ins_brown_4_white_4_06
H_mast_ins_brown_4_white_4_07
H_mast_ins_brown_4_white_4_08
H_mast_ins_brown_4_white_4_09
H_mast_ins_brown_4_white_4_10
H_mast_ins_brown_4_white_4_11
12
XYZ(-59.35006999999998 -664.0132600000001 0.6259899999999994) Yaw(-191.12001)
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [22]:
# control.moveToPosition(client, masts[0], 0, NORTH_EAST_POS, 6)
# control.moveToPosition(client, masts[0], 0, NORTH_WEST_POS, INSPECTION_SPEED)

In [23]:
#client.simSetCameraOrientation("0", airsim.to_quaternion(-0.5235, 0, 0)) #radians
#home_location = client.simGetGroundTruthKinematics().position

name = os.path.join(FLYING_LOG_DIR, "logs.txt")
f = open(name, "a+")

recording = False

for i in range(0, 500):

    _, _, cur_yaw_rad = getPitchRollYawInRad()
    cur_yaw_degree = toDegree(cur_yaw_rad)
    cur_z = getZ()
    f.write('{}, cur_yaw {:.3f}, cur_z {:.3f}\n'.format(i, cur_yaw_degree, cur_z))
    
    responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])
    height_prob, offset_prob, view_prob  = transform_input(responses, i, f, recording)   
    
    # detect mast, line
        # show detection box which requires a new Airsim's API
    
    
    
    # navigate
    new_z = 0.35*(height_prob[0] - height_prob[2]) # down - up
    latoff_angle = 10 * (offset_prob[0] - offset_prob[2]) # left - right
    
    view_angle = 0
    headless = False 
    if (view_prob[1] > 0.95):
        headless = True       
    else:
        view_angle = 10 * (view_prob[0] - view_prob[2]) # left - right
    
    new_yaw_degree =  view_angle + latoff_angle 
    
    if (recording):
        f.write('{}, added_yaw {:.3f}, added_z {:.3f}\n'.format(i, new_yaw_degree, new_z))
        f.write('{}, view_angle {:.3f}, latoff_angle {:.3f}\n'.format(i, view_angle, latoff_angle))
        
    new_yaw_rad = toRad(new_yaw_degree)
    
    velocity = 0.25 + 0.25*offset_prob[1] + 0.25*view_prob[1]
    if (new_yaw_degree > 11 or new_yaw_degree < -11):
        if (recording):
            f.write('****turning with a BIG ANGLE\n')
        #moveByYawPitch(0, cur_z, 0)
        client.rotateToYawAsync(cur_yaw_degree + new_yaw_degree, 1, 1).join()
    
    _, _, cur_yaw_rad = getPitchRollYawInRad()
    moveByYawPitch(cur_yaw_rad, new_yaw_rad, cur_z - new_z, velocity, headless, file=f, recording=True)


#work around to make drone stop
moveByYawPitch(0, 0, getZ(), 0)
client.hoverAsync().join()

f.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


KeyboardInterrupt: 

In [21]:
name = os.path.join(FLYING_LOG_DIR, "logs.txt")
f = open(name, "a+")

recording = True

for i in range(1300, 1800):

    _, _, cur_yaw = getPitchRollYawInRad()
    cur_z = getZ()
    f.write('{}, cur_yaw {:.3f}, cur_z {:.3f}\n'.format(i, toDegree(cur_yaw), cur_z))
    
    responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])
    height_prob, offset_prob, view_prob  = transform_input(responses, i, f, recording)   
    
    # detect mast, line
        # show detection box which requires a new Airsim's API
    
    
    
    # navigate
    new_z = 0.35*(height_prob[0] - height_prob[2]) # down - up
    latoff_angle = 10 * (offset_prob[0] - offset_prob[2]) # left - right
    view_angle = 10 * (view_prob[0] - view_prob[2]) # left - right
    new_yaw_degree =  view_angle + latoff_angle 
    
    if (recording):
        f.write('{}, added_yaw {:.3f}, added_z {:.3f}\n'.format(i, new_yaw_degree, new_z))
        f.write('{}, view_angle {:.3f}, latoff_angle {:.3f}\n'.format(i, view_angle, latoff_angle))
        
    new_yaw = toRad(new_yaw_degree)
    
    velocity = 0.25 + 0.25*offset_prob[1] + 0.25*view_prob[1]
    if (new_yaw_degree > 11 or new_yaw_degree < -11):
        if (recording):
            f.write('****turning with a BIG ANGLE\n')
        #moveByYawPitch(0, cur_z, 0)
        client.rotateToYawAsync(toDegree(cur_yaw + new_yaw), 1, 1).join()
    
    _, _, cur_yaw = getPitchRollYawInRad()
    moveByYawPitch(cur_yaw + new_yaw, cur_z - new_z, velocity, f)


#work around to make drone stop
moveByYawPitch(0, getZ(), 0, f)
client.hoverAsync().join()

f.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


mastNo = 6
control.moveToPosition(client, masts[firstInd + mastNo - 1], masts[firstInd + mastNo - 1].yaw, NORTH_SOUTH_DRT, SOUTH_WEST_POS, 2)
control.moveToPosition(client, masts[firstInd + mastNo - 1], masts[firstInd + mastNo - 1].yaw, WEST_EAST_DRT, SOUTH_EAST_POS)

In [ ]:
# client.moveByAngleZAsync(pitch=1, roll=0, yaw=1)
# client.simGetObjectPose("H_mast_ins_brown_4_white_0")

### 